# Liz Danzico: VP of Design @Microsoft and Long Beloved Design Luminary

### “What I need is the ability to watch the motion and noise of people. Just the observation of something happening is all I need in order to create something. Without that, I have nothing.” —via [The Great Discontent](https://thegreatdiscontent.com/interview/liz-danzico/)

### With Liz, we examine how AI agents can be used to help people write better (often **longer**) prompts. Learn more about Liz Danzico and the evolution of Bing and Copilot via [The New Yorker](https://www.newyorker.com/culture/infinite-scroll/bing-ai-and-the-dawn-of-the-post-search-internet)

## 🔥 Let's get the required packages

In [ ]:
#!import ../config/Settings.cs 
#!import ../config/Utils.cs

#r "nuget: Microsoft.SemanticKernel, 1.4.0"
#r "nuget: Microsoft.SemanticKernel.Experimental.Agents, 1.4.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.4.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.4.0-alpha"
#r "nuget: Microsoft.Extensions.Logging.Console, 8.0.0"
#r "nuget: YamlDotNet, 13.7.1"

In [ ]:
#!import ../config/Settings.cs
#!import ../config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Planning.Handlebars;
using Microsoft.Extensions.Logging;
using Kernel = Microsoft.SemanticKernel.Kernel;

Kernel kernel;

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

## 🧑‍🍳 Let's get cooking on prompting with agents

In [ ]:
#r "nuget: Microsoft.CognitiveServices.Speech, 1.34.1"
#r "nuget: NetCoreAudio, 1.8.0"
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.EnvironmentVariables, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.UserSecrets, 8.0.0"

#!import ../config/Utils.cs 
#!import ../config/AzureSpeech.cs

Utils.LoadEnvFile();

string subscriptionKey = Environment.GetEnvironmentVariable("AZURE_SPEECH_KEY");
string subscriptionRegion = Environment.GetEnvironmentVariable("AZURE_SPEECH_REGION");

var speechService = new SpeechRecognitionService(subscriptionKey, subscriptionRegion);

### 🎤 Let's listen up

In [ ]:
string recognizedText = await speechService.RecognizeOnceAsync();
Console.WriteLine($"Transcribed: {recognizedText}");

### 🗣️ And let's speak up

In [ ]:
#pragma warning disable SKEXP0101

await speechService.SynthesizeSpeechAsync(recognizedText);

## 🔥 We fire up Semantic Kernel's Experimental Agents ...

In [ ]:
#r "nuget: Microsoft.SemanticKernel.Experimental.Agents, 1.4.0-alpha"
#r "nuget: YamlDotNet, 13.7.1"

using System;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;
using Microsoft.SemanticKernel.Experimental.Agents;

const string OpenAIFunctionEnabledModel = "gpt-4-1106-preview";


In [ ]:
using System.IO;
using YamlDotNet.Serialization;
using YamlDotNet.Serialization.NamingConventions;

using System.IO;
using YamlDotNet.Serialization;
using YamlDotNet.Serialization.NamingConventions;
using System;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;
using Microsoft.SemanticKernel.Experimental.Agents;

#pragma warning disable SKEXP0101

public class Personay
{
    public string Name { get; set; }
    public string Instructions { get; set; }
    public string Description { get; set; }
}

public class NameGenerator
{
    private List<string> names;
    private int currentIndex = -1;

    public NameGenerator()
    {
        // Initialize the list with ungendered names
        names = new List<string>
        {
            "Alex", "Jordan", "Taylor", "Morgan", "Casey",
            "Riley", "Jamie", "Avery", "Reese", "Skyler",
            "Quinn", "Peyton", "Cameron", "Sawyer", "Drew",
            "Charlie", "Emerson", "Dakota", "Parker", "Sidney"
        };
    }

    public string GetNextName()
    {
        // Increment the index and reset if it exceeds the list count
        currentIndex = (currentIndex + 1) % names.Count;
        return names[currentIndex];
    }
}

 // Track agents for clean-up
static readonly Dictionary<string, IAgent> s_agents = new();

IAgentThread? s_currentThread = null;

async Task<IAgent> CreateAgentAsync(string name, string instructions, string description)
{
    var agent = await new AgentBuilder()
                    .WithOpenAIChatCompletion(OpenAIFunctionEnabledModel, apiKey)
                    .WithInstructions(instructions)
                    .WithName(name)
                    .WithDescription(description)
                    .BuildAsync();

    return Track(name, agent);
}

async Task CleanUpAsync()
{
    Console.WriteLine("🧽 Cleaning up ...");

    if (s_currentThread != null)
    {
        Console.WriteLine("Thread going away ...");
        s_currentThread.DeleteAsync();
        s_currentThread = null;
    }
    
    if (s_agents.Any())
    {
        Console.WriteLine("Agents going away ...");
        await Task.WhenAll(s_agents.Values.Select(agent => agent.DeleteAsync()));
        s_agents.Clear();
    }
}

IAgent Track(string name, IAgent agent)
{
    s_agents[name] = agent; // Add or update the agent in the dictionary
    return agent;
}



### 🐣 Let's hatch the agent(s)

In [ ]:
List<string> personasAvailable = [
    "SearchEngineRobot",
    "PromptTeacher",
    "PromptWriter"];

List<(string Name, string Instructions, string Description)> agentInfo = new();
NameGenerator nameGenerator = new NameGenerator();

var rollcall = "This is the rollcall for the prompt helpers:\n";

foreach (var (a, i) in personasAvailable.Select((value, idx) => (value, idx + 1)))
{
    var yaml = File.ReadAllText($"../agents/{a}.yaml");
    var deserializer = new DeserializerBuilder()
        .WithNamingConvention(CamelCaseNamingConvention.Instance) // Use camel case naming convention
        .Build();
    var p = deserializer.Deserialize<Personay>(yaml);
    string fakeName = nameGenerator.GetNextName();
    string desc = $"{p.Name}: {p.Description}";
    string instr = p.Instructions;
    agentInfo.Add((fakeName, instr, desc));
    Console.WriteLine($"Agent defined: {fakeName} --> {desc}");
    rollcall += $"- {i}) {fakeName} is the {desc}\n";
}

foreach (var (name, instructions, description) in agentInfo)
{
    await CreateAgentAsync(name, instructions, description);
}

Console.WriteLine(rollcall);
s_agents

## 🏃 The moment of prompting by text or by voice

### ⌨️ By text ...

In [ ]:
var myPrompt = @"
    I want to get the files from a server that 
    I own and send them to a client.
";

### 🗣️ By voice ...

In [ ]:
string myPrompt = await speechService.RecognizeOnceAsync();
Console.WriteLine($"My prompt: {recognizedText}");

## 1️⃣ 🔍 Alex please

In [ ]:
#pragma warning disable SKEXP0101

IAgentThread? thread = null;

thread = await s_agents["Alex"].NewThreadAsync();

// write a function that abstracts the below

async Task<string>GoTellAgent(IAgentThread thread, string agentName, string message)
{
    Console.WriteLine($"🥸 Asking {agentName} ({s_agents[agentName].Description}) ...");
    await thread.AddUserMessageAsync(message);
    var agentMessages = await thread.InvokeAsync(s_agents[agentName]).ToArrayAsync();
    return agentMessages[0].Content;
}

Console.WriteLine(
    Utils.WordWrap(
        GoTellAgent(thread, "Alex", myPrompt).Result, 80));

## 2️⃣ 🧑‍🏫 Jordan please

In [ ]:
Console.WriteLine(
    Utils.WordWrap(
        GoTellAgent(thread, "Jordan", myPrompt).Result, 80));

## 3️⃣ ✍️ Taylor please

In [ ]:
Console.WriteLine(
    Utils.WordWrap(
        GoTellAgent(thread, "Taylor", myPrompt).Result, 80));

## 🧼 Let's do some quick agents cleanup

In [ ]:
await CleanUpAsync()